<a href="https://colab.research.google.com/github/teim-projects/Group3_Fouling_Factor_Prediction_in_Heat_Exchanger/blob/main/FuturePrediction_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
filename = "/content/HeatExchanger_data.csv"
df = pd.read_csv(filename)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   date                            144 non-null    object 
 1   Crude Temp In                   144 non-null    float64
 2   Crude Temp Out                  144 non-null    float64
 3   CDU1 rate  
(KBPD)              144 non-null    float64
 4   Crude Flow rate (kg/hr)         144 non-null    float64
 5   Kero Temp In                    144 non-null    float64
 6   Kero Temp Out                   144 non-null    float64
 7   Kero Flow m3/hr                 144 non-null    float64
 8   Furnace inlet temp (deg C)      144 non-null    float64
 9   Crude Temp Increase             144 non-null    float64
 10   Kero Temp decrease             144 non-null    float64
 11  Q (heat exchanged) MW           144 non-null    float64
 12  Hot-in - Cold-out               144 

In [ ]:
df['date '] = pd.to_datetime(df['date '])
df.set_axis(df['date '], inplace=True)
df.drop(columns=['Crude Temp In ', 'Crude Temp Out ', 'CDU1 rate  \n(KBPD)', 'Crude Flow rate (kg/hr)', 'Kero Temp In ', 'Kero Temp Out ', 'Kero Flow m3/hr ', 'Furnace inlet temp (deg C)', 'Crude Temp Increase', ' Kero Temp decrease', 'Q (heat exchanged) MW', 'Hot-in - Cold-out', 'Hot-out -Cold-in', 'LMTD', 'U transfer rate', 'Cummulative Flow Tones per day'], inplace=True)

<ipython-input-3-2eaade8c975d>:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['date '] = pd.to_datetime(df['date '])
<ipython-input-3-2eaade8c975d>:2: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(df['date '], inplace=True)


In [ ]:
close_data = df['Fouling Resistance'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df['date '][:split]
date_test = df['date '][split:]

print(len(close_train))
print(len(close_test))

115
29


In [ ]:
look_back = 15

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 1000
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/1000


<ipython-input-6-b7138a8fd00f>:14: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=num_epochs, verbose=1)


5/5 [==============================] - 2s 12ms/step - loss: 1.5000e-05
Epoch 2/1000
5/5 [==============================] - 0s 11ms/step - loss: 9.2189e-06
Epoch 3/1000
5/5 [==============================] - 0s 8ms/step - loss: 1.6427e-06
Epoch 4/1000
5/5 [==============================] - 0s 8ms/step - loss: 3.2611e-06
Epoch 5/1000
5/5 [==============================] - 0s 8ms/step - loss: 8.9554e-07
Epoch 6/1000
5/5 [==============================] - 0s 8ms/step - loss: 9.4237e-07
Epoch 7/1000
5/5 [==============================] - 0s 8ms/step - loss: 4.1098e-07
Epoch 8/1000
5/5 [==============================] - 0s 8ms/step - loss: 5.3946e-07
Epoch 9/1000
5/5 [==============================] - 0s 8ms/step - loss: 3.1778e-07
Epoch 10/1000
5/5 [==============================] - 0s 10ms/step - loss: 2.8859e-07
Epoch 11/1000
5/5 [==============================] - 0s 8ms/step - loss: 3.0421e-07
Epoch 12/1000
5/5 [==============================] - 0s 9ms/step - loss: 3.1878e-07
Epoch 13/10

In [ ]:
import plotly.graph_objects as go
prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))


trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Heat exchanger data",
    xaxis = {'title' : "date"},
    yaxis = {'title' : "Fouling resistance"}
)
fig = go.Figure(data=[trace2, trace3], layout=layout)
fig.show()

<ipython-input-7-8a4f0c8d6805>:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  prediction = model.predict_generator(test_generator)


In [ ]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['date '].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

1/1 [==============================] - 0s 21ms/step


In [ ]:
forecast_dates

[Timestamp('2012-12-31 00:00:00', freq='D'),
 Timestamp('2013-01-01 00:00:00', freq='D'),
 Timestamp('2013-01-02 00:00:00', freq='D'),
 Timestamp('2013-01-03 00:00:00', freq='D'),
 Timestamp('2013-01-04 00:00:00', freq='D'),
 Timestamp('2013-01-05 00:00:00', freq='D'),
 Timestamp('2013-01-06 00:00:00', freq='D'),
 Timestamp('2013-01-07 00:00:00', freq='D'),
 Timestamp('2013-01-08 00:00:00', freq='D'),
 Timestamp('2013-01-09 00:00:00', freq='D'),
 Timestamp('2013-01-10 00:00:00', freq='D'),
 Timestamp('2013-01-11 00:00:00', freq='D'),
 Timestamp('2013-01-12 00:00:00', freq='D'),
 Timestamp('2013-01-13 00:00:00', freq='D'),
 Timestamp('2013-01-14 00:00:00', freq='D'),
 Timestamp('2013-01-15 00:00:00', freq='D'),
 Timestamp('2013-01-16 00:00:00', freq='D'),
 Timestamp('2013-01-17 00:00:00', freq='D'),
 Timestamp('2013-01-18 00:00:00', freq='D'),
 Timestamp('2013-01-19 00:00:00', freq='D'),
 Timestamp('2013-01-20 00:00:00', freq='D'),
 Timestamp('2013-01-21 00:00:00', freq='D'),
 Timestamp

In [ ]:
forecast

array([0.0091254 , 0.00841291, 0.00827587, 0.00816173, 0.00806664,
       0.00798723, 0.00792085, 0.00786515, 0.00781885, 0.00777997,
       0.00774726, 0.00772009, 0.00769725, 0.00767813, 0.00766221,
       0.00764916, 0.00763823, 0.00762952, 0.00762232, 0.00761636,
       0.00761144, 0.00760737, 0.007604  , 0.00760122, 0.00759892,
       0.00759702, 0.00759545, 0.00759415, 0.00759307, 0.00759219,
       0.00759146])

In [ ]:
df=pd.DataFrame({'Predicted':forecast})
df.sort_index()

,Predicted
0,0.009125
1,0.008413
2,0.008276
3,0.008162
4,0.008067
5,0.007987
6,0.007921
7,0.007865
8,0.007819
9,0.007780
